Install Dependencies

In [1]:
%pip install gymnasium[accept-rom-license]

  Using cached AutoROM-0.4.2-py3-none-any.whl (16 kB)
  Attempting uninstall: autorom
    Found existing installation: AutoROM 0.6.1
    Uninstalling AutoROM-0.6.1:
      Successfully uninstalled AutoROM-0.6.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install gymnasium[atari]

  Using cached Shimmy-0.2.1-py3-none-any.whl (25 kB)
  Attempting uninstall: shimmy
    Found existing installation: Shimmy 1.1.0
    Uninstalling Shimmy-1.1.0:
      Successfully uninstalled Shimmy-1.1.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install stable-baselines3[extra]

  Obtaining dependency information for shimmy[atari]~=1.1.0 from https://files.pythonhosted.org/packages/d5/fb/083e36bbcf325f6304bbeb2278b102c4ac8e87eb1ca771780f64decbb2f1/Shimmy-1.1.0-py3-none-any.whl.metadata
  Using cached Shimmy-1.1.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached AutoROM-0.6.1-py3-none-any.whl (9.4 kB)
Using cached Shimmy-1.1.0-py3-none-any.whl (37 kB)
  Attempting uninstall: shimmy
    Found existing installation: Shimmy 0.2.1
    Uninstalling Shimmy-0.2.1:
      Successfully uninstalled Shimmy-0.2.1
  Attempting uninstall: autorom
    Found existing installation: AutoROM 0.4.2
    Uninstalling AutoROM-0.4.2:
      Successfully uninstalled AutoROM-0.4.2
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


Import Dependencies

In [5]:
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C


Constants

In [18]:
ENV_NAME = 'ALE/Adventure-v5'
ENV_N_STACKS = 9
ENV_SEED = 0
ENV_RENDER_MODE = 'human'

CALLBACK_CURRENT_DIR = os.getcwd()
CALLBACK_CHECKPOINT_DIR = f'{CALLBACK_CURRENT_DIR}\models'
CALLBACK_LOG_DIR = f'{CALLBACK_CURRENT_DIR}\logs'
CALLBACK_CHECK_FREQ = 10000

VERBOSE = 1

ALGORITHM_BEST_MODEL_NAME = 'best_model.zip'
ALGORITHM_POLICY = 'CnnPolicy'
ALGORITHM_DEVICE = 'cuda'
ALGORITHM_TOTAL_TIMESTEPS = 1000

ALGORITHM_LOAD_MODEL = False
ALGORITHM_NEW_MODEL = True
ALGORITHM_RENDER_MODEL = False
ALGORITHM_RENDER_EPISODES = 10

MESSAGE_RENDERING_MODEL = 'RENDERING MODEL'
MESSAGE_LOADING_MODEL = 'LOADING MODEL'
MESSAGE_TRAINING_NEW_MODEL = 'NEW MODEL'


Creating the Env

In [7]:
vec_env = make_atari_env(ENV_NAME, n_envs=ENV_N_STACKS, seed=ENV_SEED)
vec_env = VecFrameStack(vec_env, n_stack=ENV_N_STACKS)

Creating Callback

In [8]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=VERBOSE):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, '{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

callback = TrainAndLoggingCallback(check_freq=CALLBACK_CHECK_FREQ, save_path=CALLBACK_CHECKPOINT_DIR)

Loading, Training and Rendering

In [19]:
model = None

if ALGORITHM_RENDER_MODEL is True and ALGORITHM_LOAD_MODEL is True:
    print(MESSAGE_RENDERING_MODEL)

    if ALGORITHM_LOAD_MODEL is True:
        model = A2C.load(ALGORITHM_BEST_MODEL_NAME, env=vec_env)

    for ep in range(ALGORITHM_RENDER_EPISODES):
        obs = vec_env.reset()
        for ep in range(ALGORITHM_RENDER_EPISODES):
            action, _ = model.predict(obs, deterministic=False)
            obs, rewards, dones, info = vec_env.step(action)
            vec_env.render(ENV_RENDER_MODE)
    vec_env.close()

elif ALGORITHM_LOAD_MODEL is True:
    print(MESSAGE_LOADING_MODEL)
    model = A2C.load(ALGORITHM_BEST_MODEL_NAME, env=vec_env)
    model.learn(total_timesteps=ALGORITHM_TOTAL_TIMESTEPS, callback=callback) 

elif ALGORITHM_NEW_MODEL is True:
    print(MESSAGE_TRAINING_NEW_MODEL)
    model = A2C(ALGORITHM_POLICY, vec_env, tensorboard_log = CALLBACK_LOG_DIR, verbose=VERBOSE, device=ALGORITHM_DEVICE)
    model.learn(total_timesteps=ALGORITHM_TOTAL_TIMESTEPS, callback=callback) 

NEW MODEL
Using cuda device
Wrapping the env in a VecTransposeImage.


FailedPreconditionError: c:\Users\Henrique Bidarte\Documents\Henrique\Programação\git\sb3-gymnasium-samples\test\logs\A2C_1 is not a directory